# Testing Xception

In [ ]:
import tensorflow as tf
import data.imageReading as ir
from fairness import fairnessMetrics as fm
from model import model as m
from model import utils as utils
from model import biasMitigation as mit
from model import evaluation as ev
from tuning import callbacks as cb
import datetime
import os
import numpy as np

In [ ]:
image_size = (299,299)
batch_size = 128
# Create new datasets
#(ds_train, train_batches, ds_val, val_batches, ds_test, test_batches, count_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces", image_size, batch_size, preprocess_input)
#(ffds_train, fftrain_batches, ffds_val, ffval_batches, ffds_test, fftest_batches, ffcount_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/FairFace", image_size, batch_size, preprocess_input)

# Load datasets from current _split folders
(ds_train, train_batches, ds_val, val_batches, ds_test, test_batches, count_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces_split", image_size, batch_size, None, False, False)
(ffds_train, fftrain_batches, ffds_val, ffval_batches, ffds_test, fftest_batches, ffcount_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/FairFace_split", image_size, batch_size, None, False, False)


## Find learning rate

In [ ]:
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = m.build_model(False, (299,299), "Xception")
    metric_list = m.metrics_list()

model.compile(optimizer=tf.keras.optimizers.Adam(), 
                           loss="binary_crossentropy", 
                           metrics="accuracy")


In [ ]:
epochs = 200
train_input = []
train_output = []
nr_batches = 10

model.fit(ds_train, steps_per_epoch=nr_batches, callbacks=[cb.LerningRateCallback(100, nr_batches)], epochs=epochs, verbose=2)

# FairFace

## Test Xception FairFace (unweighted) 

In [ ]:
epochs = 20
class_weight = mit.findClassWeights(fftrain_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = m.build_model(False, (299,299), "Xception")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)


utils.train_model(model, epochs, ffds_train, fftrain_batches, ffds_val, ffval_batches, None, False)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_FairFace"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/Xception_FairFace.h5")

In [ ]:
test_predict, test_labels, dir_name = ev.testModel(model, ffds_test, fftest_batches, dir_name)
ev.testModelWithThresholdChange(model, ffds_val, ffval_batches, test_predict, test_labels, dir_name)

## Test Xception FairFace (weighted)

In [ ]:
epochs = 20
class_weight = mit.findClassWeights(fftrain_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = m.build_model(False, (299,299), "Xception")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)


utils.train_model(model, epochs, ffds_train, fftrain_batches, ffds_val, ffval_batches, class_weight)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_FairFace_re"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/Xception_FairFace_re.h5")

In [ ]:
test_predict, test_labels, dir_name = ev.testModel(model, ffds_test, fftest_batches, dir_name)
ev.testModelWithThresholdChange(model, ffds_val, ffval_batches, test_predict, test_labels, dir_name)

## Test Xception FairFace Augmented (unweighted)

In [ ]:
epochs = 20
class_weight = mit.findClassWeights(fftrain_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = m.build_model(True, (299,299), "Xception")
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )


model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)


utils.train_model(model, epochs, ffds_train, fftrain_batches, ffds_val, ffval_batches, None, False)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_FairFace_aug"
os.mkdir(dir_name)

utils.saveModel(model, dir_name + "/Xception_FairFace_aug.h5")

In [ ]:
test_predict, test_labels, dir_name = ev.testModel(model, ffds_test, fftest_batches, dir_name)
ev.testModelWithThresholdChange(model, ffds_val, ffval_batches, test_predict, test_labels, dir_name)

# Transfer Learning

## Transfer learning without augmentation and reweighting

In [ ]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Total number of layers: 137
utils.freezeCertainLayers(model, 134)    
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, None, False)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_transfer"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name)
utils.saveModel(model, dir_name_transfer + "/Xception_transfer.h5")

In [ ]:
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        5e-4,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )

history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (299,299), None, 5, 20)

In [ ]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Xception"
os.mkdir(dir_name)

ev.evaluateCrossValidation(dir_name, history)

## Transfer learning with augmentation and without reweighting

In [ ]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Total number of layers: 137 
utils.freezeCertainLayers(model, 134)  
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, None, False)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_transfer_aug"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name)
utils.saveModel(model, dir_name_transfer + "/Xception_transfer_aug.h5")

In [ ]:
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        5e-4,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )

history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (299,299), None, 5, 20)

In [ ]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Xception_aug"
os.mkdir(dir_name)

ev.evaluateCrossValidation(dir_name, history)

## Transfer learning without augmentation and with reweighting

In [ ]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace_re.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Total number of layers: 137    
utils.freezeCertainLayers(model, 134)   
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_transfer_re"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name)
utils.saveModel(model, dir_name_transfer + "/Xception_transfer_re.h5")

In [ ]:
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace_re.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        5e-4,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )

history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (299,299), class_weight, 5, 20)

In [ ]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Xception_re"
os.mkdir(dir_name)

ev.evaluateCrossValidation(dir_name, history)

## Transfer learning with augmentation and reweighting

In [ ]:
epochs = 20
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-4,
        decay_steps=100000,
        decay_rate=0.96,
    )

# Total number of layers: 137    
utils.freezeCertainLayers(model, 134)  
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight)

print("Make folder...")
dir_name_transfer = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Xception_transfer_re_aug"
os.mkdir(dir_name_transfer)

print("Name directory: " + dir_name)
utils.saveModel(model, dir_name_transfer + "/Xception_transfer_re_aug.h5")

In [ ]:
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, test_batches, dir_name)
ev.testModelWithThresholdChange(model, ds_val, val_batches, test_predict, test_labels, dir_name)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
# Choose correct folder name
dir_name = "Test"

with strategy.scope():
    # Choose the correct FairFace model
    model = utils.loadModel(dir_name + "/Xception_FairFace_aug.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        5e-4,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True
    )

history = ev.kfoldCrossValidation('/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Female/*.*', 
                      '/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces/Male/*.*',
                      model, metric_list, lr_schedule, (299,299), class_weight, 5, 20)

In [ ]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_kFoldCrossValidation_Xception_re_aug"
os.mkdir(dir_name)

ev.evaluateCrossValidation(dir_name, history)